# Text Processing 

We will work with unstructured data.
In particular, we will work with text data to build a language model (LM). 

LM is a probability distribution $p(\cdot)$ over sequence of words.
For example, given text $T=(w_1\ w_2\ \dots \ w_{|T|})$, where $w_1,\ w_2\, \dots \ w_{|T|}$ is sequence of words and $|T|$ is the number of words in the sequence, LM assigns it probability as follows:
$p(T)=p(w_1\ w_2\ \dots \ w_{|T|})$.

LM is employed in many applications such as automatic speech recognition (ASR), machine translation (MT), image captioning, text generation and so on. For example, LM is used in ASR to assess correctness of generated output transcripts.

Let's assume our ASR system generated two possible transcripts for some given speech signal: "recognize speech" and "wreck a nice beach".
These two phrases sound similar, but the first one is more likely to be linguistically correct.
We can use the probability of each transcript to determine the final output of ASR.
The probabilistic model (LM) should assign a higher probability score to the correct answer:

$p(``\text{recognize speech}") > p(``\text{wreck a nice beach}")$

## Reading a text file
We will use Penn Tree Bank ([PTB](https://catalog.ldc.upenn.edu/LDC99T42)) dataset as our text data in this assignment.

In [1]:
# Reading a file into the string
with open('ptb.train.txt') as f:
   raw_data = f.read()

print("Type:",type(raw_data))     # type of the data
print("Size:",str(len(raw_data))) # number of characters
raw_data[965:1156]                # print a small snippet of the data ("\n" is a symbol for new line, it indicates the sentence boundaries)

Type: <class 'str'>
Size: 5101618


"\n although preliminary findings were reported more than a year ago the latest results appear in today 's new england journal of medicine a forum likely to bring new attention to the problem \n"

## Text preprocessing
Take a look at the PTB dataset. It is already pre-processed, i.e. words are tokenized, all letters are lowercased, numbers are replaced with the "N", some non-alphanumeric characters are removed and etc. We need to apply a few small changes for our assignment. Please note, only the most frequent 9,999 words are kept in the dataset, the rest of the words are replaced with the special <"unk>" symbol. This is done to reduce memory and computation requirements.   


The PTB dataset has been pre-processed. Nevertheles, let's learn some basic text pre-processing commands.

To lowercase a given string, you can use `str.lower()` command.

To filter out the non-alphanumeric characters (e.g. comma(,), question mark(?), colon(:), dash(-), dollar sign($), equal symbol(=), plus sign(+), apostrophes(') and etc.) and split the text into the words, you can use python regular expression operations (check the [python documentation](https://docs.python.org/3/library/re.html) to learn more).

In [2]:
# Split the raw data into sentences and save them into a list (delimiter for sentence boundaries: '\n')
data = raw_data.split('\n')
#data = re.split('\n', raw_data)  # you can also use regular expressions
data = [sent.strip() for sent in data if sent != '']  # remove empty sentences, see "list comprehension" 

print("Type: ", type(data))   # type of the data
print("Size: ", len(data))    # number of sentences
print("Sentence: " + data[2]) # print a sentence, try to print another sentence (symbol '<unk>' corresponds to unknown word)

Type:  <class 'list'>
Size:  42068
Sentence: mr. <unk> is chairman of <unk> n.v. the dutch publishing group


## Word counting

We will create a dictionary which contains a word as a key and its frequency as a value, eg. 'judge': 262



In [3]:
dictionary = dict()  


In [4]:
for sent in data:
    for word in sent.split():
        if word in dictionary:
            dictionary[word] += 1
        else:
            dictionary[word] = 1


In [5]:
for sent in data:
    for word in sent.split():
        dictionary[word] = dictionary.get(word,0) +1


In [6]:
# Check the dictionary
print("Size: "+str(len(dictionary)))  # number of unique words
print(dictionary['judge'])            # try other words
print(dictionary['<unk>'])            # try other words

Size: 9999
524
90040


In [7]:
# #Solution1
# dictionary = dict()
# for sent in data:
#     for token in sent.split():
#         if token not in dictionary:
#             dictionary[token] = 1
#         else:
#             dictionary[token] += 1

In [8]:
#Solution2
dictionary = dict()
for sent in data:
    for token in sent.split():
        dictionary[token] = dictionary.get(token,0) + 1

## Building Unigram Language Model (probabilistic model)

To build a probabilistic model of a text, we can employ the frequentist approach, i.e. use relative frequency to estimate probability scores.
Suppose we want to estimate probability of a sentence "how are you", then we need to count how many times it appears in the dataset. This approach is infeasible for long and/or complex sentences that might be absent or appear a few times in the dataset.

Computing probability of short phrase is easy (click [here](https://books.google.com/ngrams/graph?content=How+are+you&year_start=1800&year_end=2000&corpus=15&smoothing=3&share=&direct_url=t1%3B%2CHow%20are%20you%3B%2Cc0#t1%3B%2CHow%20are%20you%3B%2Cc0)).
However, for a longer sentence it is difficult (click [here](https://books.google.com/ngrams/graph?content=I+met+my+friends+Madina+and+Yerbolat&year_start=1800&year_end=2000&corpus=15&smoothing=3&direct_url=))

#### Review: [Chain rule](https://en.wikipedia.org/wiki/Chain_rule_(probability))
To circumvent the problem mentioned above, we will employ the chain rule to break down the joint probability of all words in a sentence into the sequence of conditional probabilities as follows:

$p(T)=p(w_1\ w_2\ \dots\ w_{|T|}) = p(w_1)\ p(w_2 | w_1)\ p(w_3 | w_1\ w_2)\ \dots \ p(w_{|T|} | w_1 \dots w_{|T|-1}) = p(w_1)\prod^{|T|}_{i=2}p(w_i|w_1\ \dots\ w_{i-1})$

where $p(w_3 | w_1\ w_2)$ - is the probability of word $w_3$ given that the preceeding two words in the sentence were $w_1$ and $w_2$.

For example, if $T= ``How\ are\ you"$, then $w_1=``How"$, $w_2 = ``are"$, and $w_3 = ``you"$, and the probability of having such sentence is
$p(``How\ are\ you")= p(``How")\ p(``are"| ``How")\ p(``you" | ``How" \ ``are")$

The conditional probabilities are estimated as follows:

$p(w_i|w_1\ \dots\ w_{i-1})=\frac{count(w_{1}\ w_{2}\ \dots \ w_{i})}{count(w_1\ w_2\ \dots\ w_{i-1})}$

Nevertheless, this is still infeasible for long sentences, because computing conditional probability of the last words still requires to count occurences of the preceeding long phrases.

The simplest way to solve this problem is to treat each word independent from each other ([Independence](https://en.wikipedia.org/wiki/Markov_property)). It is also known as [unigram language model](https://en.wikipedia.org/wiki/Language_model#Unigram):

$p(T)=p(w_1\ w_2\ \dots\ w_{|T|}) \approx p(w_1)\ p(w_2)\ p(w_3)\ \dots \ p(w_{|T|}) = \prod^{|T|}_{i=1}p(w_i)$

Then: $p(``How\ are\ you")\approx p(``How")\ p(``are")\ p(``you")$

The marginal probability of each word can be estimated as follows:

$p(w_i)=\frac{count(w_i)}{\sum count(w)} = \frac{count(w_i)}{Total\ number\ of\ words}$

## Estimating probability of a word
We will create a dictionary called **dictionary_prob** which holds the marginal probability of each word, eg. 'judge': 0.0002696424767176071

In [9]:
total_words = sum(dictionary.values())
dictionary_prob = dictionary.copy()

for word in dictionary:
    dictionary_prob[word] /= total_words

In [10]:
# check the obtained dictionary_prob
print("Total number of words: ", sum(dictionary.values()))
print("Probability of 'judge': ", dictionary_prob['judge'])
print("Probability of '<unk>': ", dictionary_prob['<unk>'])

Total number of words:  887521
Probability of 'judge':  0.0002952042824902171
Probability of '<unk>':  0.050725560296601434


In [11]:
num_of_words = sum(dictionary.values())   # total number of words
dictionary_prob = dictionary.copy()
for key in dictionary_prob:
      dictionary_prob[key] /= num_of_words

print(num_of_words)
print(dictionary['judge'])
print(dictionary_prob['judge'])

887521
262
0.0002952042824902171


## Estimating probability of a sentence
We will write a function called **compute_prob0** which takes a sentence and dictionary of marginal probabilites as input and returns its probability assuming each word is independent. For the words not present in the dictionary, probability of "'\<"unk>' symbol must be used. 


In [12]:
def compute_prob0(sentence, dictionary_prob):
    tokens = sentence.split()
    result = 1
    for token in tokens:
        if token in dictionary_prob:
            result *= dictionary_prob[token]
        else:
            result *= dictionary_prob["<unk>"]
  
    return result

In [13]:
# Check you solution
sentence="paris is a capital of france"
print(sentence)
compute_prob0(sentence, dictionary_prob)

paris is a capital of france


2.4865562113295714e-17

In [14]:
# Check you solution
sentence="mr. is chairman of n.v. the dutch publishing group"
print(sentence)
compute_prob0(sentence, dictionary_prob)

mr. is chairman of n.v. the dutch publishing group


1.579782658636421e-27

In [15]:
def compute_prob0(sentence, dictionary_prob):
    tokens = sentence.split()
    result = 1.0
    for token in tokens:
        if token not in dictionary_prob:
            result *= dictionary_prob['<unk>']
        else:
            result *= dictionary_prob[token]
    return result

In [16]:
# #Solution2
# import numpy as np
# def compute_prob0(sentence, dictionary_prob):
#     tokens = sentence.split()
#     result = []
#     for token in tokens:
#         if token not in dictionary_prob:
#             result.append(dictionary_prob['<unk>'])
#         else:
#             result.append(dictionary_prob[token])
#     return np.prod(np.array(result))


Here we have seen how to compute joint probability of a sentence using the independence assumption.
However, this assumption is too weak as words do depend on each other. In the second part, we would like to introduce a better approach using Markov chain.

**Markov chain**

[Markov chain](https://en.wikipedia.org/wiki/Markov_chain)
 is based on the following assumption: *given the present, the future does not depend on the past*.

*Zeroth order Markov chain:*

The simplest form of Markov chain is so called zeroth order Markov chain where words are completely independent from each other:

$p(w_1\ w_2\ \dots\ w_{|T|}) \approx p(w_1)\ p(w_2)\ p(w_3)\ \dots \ p(w_{|T|}) = \prod^{|T|}_{i=1}p(w_i)$

Recall that this is similar to the independence assumption covered in the first part and it is known as unigram language model.

*First order Markov chain:*

In the first order Markov chain, a probability of a word is conditioned only on the preceeding word:

$p(w_1\ w_2\ \dots\ w_{|T|}) \approx p(w_1)\ p(w_2|w_1)\ p(w_2|w_3)\ \dots \ p(w_{|T|}|w_{|T|-1}) = p(w_1)\prod^{S}_{i=2}p(w_i|w_{i-1})$

This model is known as bigram (or 2-gram) language model. The conditional probabilities are estimated as:

$p(w_i|w_{i-1})=\frac{count(w_{i-1}\ w_{i})}{count(w_{i-1})}$

Similarly, in the N-th order Markov chain, a probability of a word is conditioned only on the preceeding N words.


## Computing probability of a sentence (Bigram Language Model)
We will write a function called **compute_prob1** which takes a sentence as input (and maybe some additional arguments) and returns its probability estimated using the first order Markov chain. 

Different from the unigram language model, your function should first append *\<bos\>* and *\<eos\>* symbols to the input sentence and replace all words that are not present in the dataset with *\<unk\>* symbol. 

If the original sentence was "my name is madina", then "madina" is obviously not in the dataset (see the description of the PTB). The probability of such sentence can be computed as:

p("my name is madina") $\approx$ p("my" | "\<bos\>") * p("name" | "my") * p("is" | "name") * p("\<"unk\>" | "is") * p("\<eos\>" | "\<"unk\>")

Notice that the probability of the "\<bos\>" symbol is not used during the sentence probability estimation. It is appeneded to enable the probability computation of the first word, i.e. p("my" | "\<bos\>"). On the other hand, we must include the probability of "\<eos\>" symbol to obtain a valid probability distribution, i.e. the sum of the probabilities of all outcomes must equal 1.

In [17]:
#Write you code here
dict1 = dict()
for i in range(len(data)):
    data[i] = '<a> ' + data[i] + ' <b>'
for sent in data:
    for word in sent.split():
        if word in dict1:
            dict1[word] += 1
        else:
            dict1[word] = 1

In [18]:
bigramlist=[]
for sent in data:
    new = sent.split()
    for word in range(len(new)-1):
        bigramlist.append((new[word], new[word+1]))
dict2 = dict()
for i in bigramlist:
    if i in dict2:
        dict2[i] += 1
    else:
        dict2[i] = 1

In [19]:
def compute_prob1(sent):
    list_ = ["<a>"]
    dict_bigramLM = dict()
    result = 1
    for i in sent.split():
        if i not in dict1:
            i="<unk>"
        list_.append(i)
    list_.append("<b>")
    print(list_)
    
    ll = len(list_)
      
    for i in list(zip(list_, list_[1:])):
        dict_bigramLM[i] = dict2[i] / dict1[i[0]]
        
    for i in dict_bigramLM:
        result *= dict_bigramLM[i]
        
    return dict_bigramLM.values(), result

In [20]:
#If you completed the task correctly then your function should compute:
sent1 = "my name is madina"
print(compute_prob1(sent1))
sent2 = "hello how are you"
print(compute_prob1(sent2))
sent3 = "this is an estimate"
print(compute_prob1(sent3))

['<a>', 'my', 'name', 'is', '<unk>', '<b>']
(dict_values([0.0008082152705144053, 0.0038461538461538464, 0.06766917293233082, 0.05479078642496933, 0.07385606397156819]), 8.512130344833542e-10)
['<a>', '<unk>', 'how', 'are', 'you', '<b>']
(dict_values([0.03986402966625464, 0.000444247001332741, 0.002506265664160401, 0.0007664793050587634, 0.020253164556962026]), 6.89010961921847e-13)
['<a>', 'this', 'is', 'an', 'estimate', '<b>']
(dict_values([0.009532185984596368, 0.0808039376538146, 0.01362954886193267, 0.0002876042565429968, 0.14634146341463414]), 4.418442586905126e-10)
